#### Importación base de datos https://github.com/albertogcmr/pymongo-project
Este repositorio de Alberto explica cómo

## Proyecto MongoDB

* Importar base de datos a MongoDB
* Conexión a la base de datos con Pymongo
* Extraccion de la base de datos con Pymongo, filtrando según intereses
* Limpieza de datos con Pandas
* Columna formato geopoint 
* Guardar como Json
* Importar la base de datos a MongoDB
* Crear geoindex 
* Conexión a la nueva base de datos con Pymongo, extracción de todo.
* Consultas geoespaciales sobre tu dataframe usando $near

In [281]:
from pymongo import MongoClient
import pandas as pd

In [ ]:
# conexión con la base de datos de mongo

client = MongoClient("mongodb://localhost:27017/")
db = client.companies 

In [308]:
# Query para pedir todo
companies = db.companies.find()

In [309]:
data_companies = pd.DataFrame(companies)

In [311]:
data_companies.head(3)

,_id,acquisition,acquisitions,alias_list,blog_feed_url,blog_url,category_code,competitions,created_at,crunchbase_url,...,phone_number,products,providerships,relationships,screenshots,tag_list,total_money_raised,twitter_username,updated_at,video_embeds
0,52cdef7c4bab8bd675297d8b,None,[],Zoho ManageEngine,,,enterprise,[],2007-05-25 19:24:22,http://www.crunchbase.com/company/adventnet,...,925-924-9500,[],"[{'title': 'DHFH', 'is_past': True, 'provider'...","[{'is_past': True, 'title': 'CEO and Co-Founde...","[{'available_sizes': [[[150, 94], 'assets/imag...",,$0,manageengine,Wed Oct 31 18:26:09 UTC 2012,[]
1,52cdef7c4bab8bd675297d8c,None,[],,http://blogs.zoho.com/feed,http://blogs.zoho.com/,software,"[{'competitor': {'name': 'Empressr', 'permalin...",Fri May 25 19:30:28 UTC 2007,http://www.crunchbase.com/company/zoho,...,1-888-204-3539,"[{'name': 'Zoho Office Suite', 'permalink': 'z...",[],"[{'is_past': False, 'title': 'CEO and Founder'...",[],"zoho, officesuite, spreadsheet, writer, projec...",$0,zoho,Wed Oct 30 00:07:05 UTC 2013,"[{'embed_code': '<object width=""430"" height=""2..."
2,52cdef7c4bab8bd675297d8a,"{'price_amount': 30000000, 'price_currency_cod...",[],,http://digitalquarters.net/feed/,http://digitalquarters.net/,web,"[{'competitor': {'name': 'Wikia', 'permalink':...",2007-05-25 06:51:27,http://www.crunchbase.com/company/wetpaint,...,206.859.6300,"[{'name': 'Wikison Wetpaint', 'permalink': 'we...",[],"[{'is_past': False, 'title': 'Co-Founder and V...","[{'available_sizes': [[[150, 86], 'assets/imag...","wiki, seattle, elowitz, media-industry, media-...",$39.8M,BachelrWetpaint,Sun Dec 08 07:15:44 UTC 2013,[]


In [303]:
#Ejemplo filtro de datos con PyMongo

acquisition = db.companies.find({
    "acquisition.price_amount":{
        "$gt":10000000}}, {
    "name":1,
    "acquisition.price_amount":1, "offices":1})

In [304]:
acquisition = pd.DataFrame(acquisition)

In [337]:
#creamos columna con el formato que te pide Mongo para crear el goeindex

def geopoint(x): 
    return {"type": "Point", "coordinates": [20,20]}

In [338]:
acquisition['geo']=acquisition['offices'].apply(geopoint)

In [339]:
acquisition.head()

,_id,acquisition,name,offices,geo
0,52cdef7c4bab8bd675297d8a,{'price_amount': 30000000},Wetpaint,"[{'description': '', 'address1': '710 - 2nd Av...","{'type': 'Point', 'coordinates': [20, 20]}"
1,52cdef7c4bab8bd675297d90,{'price_amount': 625000000},Postini,"[{'description': None, 'address1': '959 Skyway...","{'type': 'Point', 'coordinates': [20, 20]}"
2,52cdef7c4bab8bd675297d92,{'price_amount': 20000000},Flektor,"[{'description': None, 'address1': '8536 Natio...","{'type': 'Point', 'coordinates': [20, 20]}"
3,52cdef7c4bab8bd675297d9a,{'price_amount': 39000000},Helio,"[{'description': None, 'address1': '10960 Wils...","{'type': 'Point', 'coordinates': [20, 20]}"
4,52cdef7c4bab8bd675297d95,{'price_amount': 29000000},StumbleUpon,"[{'description': '', 'address1': '', 'address2...","{'type': 'Point', 'coordinates': [20, 20]}"


In [323]:
#Importar archivo json a Mongo

# mongoimport --db companies --collection prueba --jsonArray oficinas.json dentro de la terminal

In [324]:
#Volvemos a conectarnos a la base de datos

db = client.companies

In [ ]:
# Cogemos todos los datos

near = db.prueba.find()
df = pd.DataFrame(near)

In [341]:
#Seleccionamos un geopoint 
test = df.iloc[0, ]
test.principal_office

{'type': 'Point', 'coordinates': [-122.247573, 37.506885]}

### Operador $near

In [331]:
#Función para encontrar puntos cercanos con $near

def geonear(geopoint, maxdistance=1000):
    return db.prueba.find({
        "principal_office":{
            "$near":{
                "$geometry":geopoint,
                "$maxDistance":maxdistance
            }}})

In [336]:
geonear(test.principal_office, 100000).count()

/miniconda3/envs/ironhack/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


180